## Zulip chat data
* [Zulip](https://zulipchat.com/) is an open source chat application that is used by many open source projects.

https://github.com/onefact/datathinking.org-codespace/blob/main/notebooks/university-of-tartu/230420-zulip-chat-duckdb-loading.ipynb is an example of how to load Zulip chat data.

## TODO
- [x] try running this example notebook
- [ ] try running this example notebook with a different Zulip chat data set updated to today's date
  - [ ] try following instructions here to download latest zulip data - https://github.com/onefact/datathinking.org-codespace/blob/main/notebooks/university-of-tartu/230316-embeddings-text-data-like-chat-logs-and-networks.ipynb
  - [ ] maybe do most of the steps here and see if we can see the latest messages?
  - [x] Follow https://zulip.com/help/export-your-organization
  - [ ] rerun this notebook to see if we could update the data


In [3]:
# smallest possible test case for SQL query using jupyter notebook cell magic commands
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:



In [24]:
import os
zulip_path = os.getcwd() + "/messages-000001.json"
print(zulip_path)

/workspaces/DataThinkingUT/messages-000001.json


In [25]:
%%sql 
SELECT UNNEST(l) FROM read_json_auto('./messages-000001.json',
               maximum_object_size=99999999) tbl(l);

,unnest(l)
0,"{'id': 299488052, 'sender': 100007, 'recipient..."
1,"{'id': 299488053, 'sender': 100007, 'recipient..."
2,"{'id': 299488054, 'sender': 100007, 'recipient..."
3,"{'id': 327255223, 'sender': 544719, 'recipient..."
4,"{'id': 327934641, 'sender': 588609, 'recipient..."
...,...
552,"{'id': 368040562, 'sender': 590536, 'recipient..."
553,"{'id': 368051747, 'sender': 587794, 'recipient..."
554,"{'id': 368053528, 'sender': 590536, 'recipient..."
555,"{'id': 368055562, 'sender': 622748, 'recipient..."


In [35]:
%%sql 
COPY (SELECT UNNEST(zerver_message) AS zerver_message 
FROM read_json_auto("{{zulip_path}}",
                    maximum_object_size=99999999)) TO './zerver_message.json';


,Count
0,557


In [36]:
%%sql 
COPY (SELECT UNNEST(zerver_usermessage) AS zerver_usermessage 
FROM read_json_auto("{{zulip_path}}",
                    maximum_object_size=99999999)) TO './zerver_usermessage.json';


,Count
0,53386


In [37]:
%%sql 
COPY (SELECT UNNEST(zerver_usermessage) AS zerver_usermessage 
FROM read_json_auto("{{zulip_path}}",
                    maximum_object_size=99999999)) TO './zerver_usermessage.parquet' (FORMAT PARQUET);


,Count
0,53386


In [38]:
%%sql 
SELECT *
FROM './zerver_usermessage.parquet'


,zerver_usermessage
0,"{'id': 8257034355, 'user_profile': 544719, 'me..."
1,"{'id': 8257034356, 'user_profile': 544719, 'me..."
2,"{'id': 8257034357, 'user_profile': 544719, 'me..."
3,"{'id': 9332100997, 'user_profile': 587794, 'me..."
4,"{'id': 9332100998, 'user_profile': 544719, 'me..."
...,...
53381,"{'id': 10548357093, 'user_profile': 590323, 'm..."
53382,"{'id': 10548357094, 'user_profile': 622452, 'm..."
53383,"{'id': 10548357095, 'user_profile': 623094, 'm..."
53384,"{'id': 10548357096, 'user_profile': 624250, 'm..."


In [39]:
%%sql 
SELECT zerver_usermessage.id, zerver_usermessage.user_profile, zerver_usermessage.message, zerver_usermessage.flags_mask
FROM './zerver_usermessage.parquet'

,id,user_profile,message,flags_mask
0,8257034355,544719,299488052,1
1,8257034356,544719,299488053,1
2,8257034357,544719,299488054,1
3,9332100997,587794,327255223,1
4,9332100998,544719,327255223,1
...,...,...,...,...
53381,10548357093,590323,368056816,0
53382,10548357094,622452,368056816,0
53383,10548357095,623094,368056816,0
53384,10548357096,624250,368056816,0


In [40]:
%%sql 
SELECT zerver_usermessage.id::UBIGINT as id, 
        zerver_usermessage.user_profile::UBIGINT as user_profile, 
        zerver_usermessage.message::UBIGINT as message, 
        zerver_usermessage.flags_mask::BOOLEAN as flags_mask
FROM './zerver_usermessage.json'


,id,user_profile,message,flags_mask
0,8257034355,544719,299488052,True
1,8257034356,544719,299488053,True
2,8257034357,544719,299488054,True
3,9332100997,587794,327255223,True
4,9332100998,544719,327255223,True
...,...,...,...,...
53381,10548357093,590323,368056816,False
53382,10548357094,622452,368056816,False
53383,10548357095,623094,368056816,False
53384,10548357096,624250,368056816,False


In [42]:
%%sql 
SELECT zerver_usermessage.id::UBIGINT as id, 
        zerver_usermessage.user_profile::UBIGINT as user_profile, 
        zerver_usermessage.message::UBIGINT as message, 
        zerver_usermessage.flags_mask::BOOLEAN as flags_mask
FROM './zerver_usermessage.json'


,id,user_profile,message,flags_mask
0,8257034355,544719,299488052,True
1,8257034356,544719,299488053,True
2,8257034357,544719,299488054,True
3,9332100997,587794,327255223,True
4,9332100998,544719,327255223,True
...,...,...,...,...
53381,10548357093,590323,368056816,False
53382,10548357094,622452,368056816,False
53383,10548357095,623094,368056816,False
53384,10548357096,624250,368056816,False


In [43]:
%%sql
SELECT  zerver_message.id::UBIGINT as id,
        zerver_message.sender::UBIGINT as sender,
        zerver_message.recipient::BIGINT as recipient,
        zerver_message.realm::UBIGINT as realm,
        zerver_message.subject::VARCHAR as subject,
        zerver_message.content::VARCHAR as content,
        zerver_message.rendered_content::VARCHAR as rendered_content,
        zerver_message.rendered_content_version::UBIGINT as rendered_content_version,
        zerver_message.date_sent::VARCHAR as date_sent,
        zerver_message.sending_client::BIGINT as sending_client,
        zerver_message.last_edit_time::VARCHAR as last_edit_time,
        zerver_message.edit_history::VARCHAR as edit_history,
        zerver_message.has_attachment::BOOLEAN as has_attachment,
        zerver_message.has_image::BOOLEAN as has_image,
        zerver_message.has_link::BOOLEAN as has_link,
        zerver_message.search_tsvector::VARCHAR as search_tsvector
FROM './zerver_message.json'

	id

,id,sender,recipient,realm,subject,content,rendered_content,rendered_content_version,date_sent,sending_client,last_edit_time,edit_history,has_attachment,has_image,has_link,search_tsvector
0,299488052,100007,924073,47304,topic demonstration,This is a message on stream #**general** with ...,"<p>This is a message on stream <a class=""strea...",1,1663549367.691131,1002,NaN,NaN,False,False,True,"'demonstrate':2,14 'demonstration':2,14 'gener..."
1,299488053,100007,924073,47304,topic demonstration,Topics are a lightweight tool to keep conversa...,<p>Topics are a lightweight tool to keep conve...,1,1663549367.706309,1002,NaN,NaN,False,False,True,'conversation':10 'demonstrate':2 'demonstrati...
2,299488054,100007,924073,47304,swimming turtles,This is a message on stream #**general** with ...,"<p>This is a message on stream <a class=""strea...",1,1663549367.719102,1002,NaN,NaN,False,True,True,"'general':9 'message':6,28 'new':17 'ply':24 '..."
3,327255223,544719,924073,47304,intros,Hi! I am Jaan. I am teaching this course and t...,<p>Hi! I am Jaan. I am teaching this course an...,1,1676123703.264337,1000,NaN,NaN,False,False,False,'agency':32 'artificial':59 'behavior':44 'clo...
4,327934641,588609,924073,47304,intros,"Hello Jaan,\nI am Nesma, a Ph.D. student at th...","<p>Hello Jaan,<br>\nI am Nesma, a Ph.D. studen...",1,1676443984.123736,1163,NaN,NaN,False,False,False,'acquire':45 'additional':70 'analysis':58 'ap...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,368040562,590536,924073,47304,main,My HW2:\n[Moonsund_Regatta_in_Estonia.png](/us...,"<p>My HW2:<br>\n<a href=""/user_uploads/47304/x...",1,1687291025.133785,1000,NaN,NaN,True,True,True,'/onefact/datathinking.org/issues/110':8 '/one...
553,368051747,587794,924073,47304,Final Project,My final project was a rushed job about:\n\n*I...,<p>My final project was a rushed job about:</p...,1,1687294428.516468,1000,1687294468.508179,"[{""user_id"":587794,""timestamp"":1687294468,""pre...",False,False,True,'/ikr503/blog.datathinking.org/blob/indrekblog...
554,368053528,590536,924073,47304,Homework critique,My critique of my own HW:\nhttps://github.com/...,"<p>My critique of my own HW:<br>\n<a href=""htt...",1,1687294959.328329,1000,NaN,NaN,False,False,True,'/onefact/datathinking.org/issues/163#issuecom...
555,368055562,622748,924073,47304,experience reports,My experience report: I had trouble setting up...,<p>My experience report: I had trouble setting...,1,1687295705.690825,1000,NaN,NaN,False,False,False,'2':106 '3':37 'assumption':118 'beliefs':120 ...


In [44]:
%%sql 
COPY
(SELECT zerver_usermessage.id::UBIGINT as id, 
        zerver_usermessage.user_profile::UBIGINT as user_profile, 
        zerver_usermessage.message::UBIGINT as message, 
        zerver_usermessage.flags_mask::BOOLEAN as flags_mask
FROM './zerver_usermessage.json'
) TO './zerver_usermessage.parquet' (FORMAT PARQUET);

,Count
0,53386


In [45]:
%%sql
COPY (
SELECT  zerver_message.id::UBIGINT as id,
        zerver_message.sender::UBIGINT as sender,
        zerver_message.recipient::BIGINT as recipient,
        zerver_message.realm::UBIGINT as realm,
        zerver_message.subject::VARCHAR as subject,
        zerver_message.content::VARCHAR as content,
        zerver_message.rendered_content::VARCHAR as rendered_content,
        zerver_message.rendered_content_version::UBIGINT as rendered_content_version,
        zerver_message.date_sent::VARCHAR as date_sent,
        zerver_message.sending_client::BIGINT as sending_client,
        zerver_message.last_edit_time::VARCHAR as last_edit_time,
        zerver_message.edit_history::VARCHAR as edit_history,
        zerver_message.has_attachment::BOOLEAN as has_attachment,
        zerver_message.has_image::BOOLEAN as has_image,
        zerver_message.has_link::BOOLEAN as has_link,
        zerver_message.search_tsvector::VARCHAR as search_tsvector
FROM './zerver_message.json'
)
TO './zerver_message.parquet' (FORMAT PARQUET);


,Count
0,557


## Visualization of the Zulip chat data

In [134]:
import vegafusion as vf
import polars as pl
import altair as alt
import altair as alt
alt.data_transformers.disable_max_rows()
alt.renderers.enable('html')

# Configure DuckDB connection
vf.runtime.set_connection("duckdb")

# Enable Mime Renderer
vf.enable(row_limit=100000000)

vegafusion.enable(mimetype='html', row_limit=100000000, embed_options=None)

In [173]:
# Load the chat data into a Polars datafram
zulip_chat = pl.read_parquet("data/datathinking.zulipchat.com/processed/zerver_message.parquet")

In [171]:
# Or, load it from the Amazon S3 public bucket for data thinking
zulip_chat = pl.read_parquet("https://public.datathinking.org/datathinking.zulipchat.com%2Fzerver_message.parquet")

In [174]:
zulip_chat

id,sender,recipient,realm,subject,content,rendered_content,rendered_content_version,date_sent,sending_client,last_edit_time,edit_history,has_attachment,has_image,has_link,search_tsvector
u64,u64,i64,u64,str,str,str,u64,str,i64,str,str,bool,bool,bool,str
299488052,100007,924073,47304,"""topic demonstr…","""This is a mess…","""<p>This is a m…",1,"""1663549367.691…",1002,null,null,false,false,true,"""'demonstrate':…"
299488053,100007,924073,47304,"""topic demonstr…","""Topics are a l…","""<p>Topics are …",1,"""1663549367.706…",1002,null,null,false,false,true,"""'conversation'…"
299488054,100007,924073,47304,"""swimming turtl…","""This is a mess…","""<p>This is a m…",1,"""1663549367.719…",1002,null,null,false,true,true,"""'general':9 'm…"
327255223,544719,924073,47304,"""intros""","""Hi! I am Jaan.…","""<p>Hi! I am Ja…",1,"""1676123703.264…",1000,null,null,false,false,false,"""'agency':32 'a…"
327934641,588609,924073,47304,"""intros""","""Hello Jaan, I …","""<p>Hello Jaan,…",1,"""1676443984.123…",1163,null,null,false,false,false,"""'acquire':45 '…"
328083032,544719,924073,47304,"""intros""","""welcome @**Nes…","""<p>welcome <sp…",1,"""1676486729.518…",1163,null,null,false,false,false,"""'intro':1 'mah…"
328216053,587794,924073,47304,"""intros""","""Hello, I am In…","""<p>Hello, I am…",1,"""1676548045.407…",1000,null,null,false,false,false,"""'analysis':44 …"
328408089,589742,924073,47304,"""intros""","""Hi, I'm Uku. I…","""<p>Hi, I'm Uku…",1,"""1676622111.241…",1000,null,null,false,false,false,"""'5':42 'ago':4…"
328418883,589761,924073,47304,"""intros""","""Hello, @**Jaan…","""<p>Hello, <spa…",1,"""1676625579.932…",1000,"""1676625630.889…","""[{""user_id"":58…",false,false,false,"""'altosaar':4 '…"


## How can we check this is the actual data? 
* look at Zulip and compare?
the most recent message:
> My experience report: Engaging with these new tools and code was very intimidating, especially trying to tackle the instructions on my own. I expected this feeling as this area is unfamiliar to me. The first obstacle was trying to get the student development pack verified. After finally figuring out the VPN issue, I still did not receive a confirmation of approval. After multiple attempts and a lot of feelings of overwhelm and stress, I decided to reach out and attend 1-1 office hours with Jaan. Through this office hour and the guidance of Jaan, it was easier to be more patient with myself, break down instructions given, navigate new tools, and troubleshoot when things didn't run smoothly. Having the guidance of the instructor the assignment seems a lot less intimidating and easier to tackle. Similarly to what Jalynn said, I did not know that my laptop had something called a terminal. The hardest part for me was understanding specific vocabulary and navigating new tools ie. knowing what buttons to click, where to find files etc.


can we find this message in the data in python?

In [149]:
zulip_chat['content'][-1]

"My experience report: Engaging with these new tools and code was very intimidating, especially trying to tackle the instructions on my own. I expected this feeling as this area is unfamiliar to me. The first obstacle was trying to get the student development pack verified. After finally figuring out the VPN issue, I still did not receive a confirmation of approval. After multiple attempts and a lot of feelings of overwhelm and stress, I decided to reach out and attend 1-1 office hours with Jaan. Through this office hour and the guidance of Jaan, it was easier to be more patient with myself, break down instructions given, navigate new tools, and troubleshoot when things didn't run smoothly. Having the guidance of the instructor the assignment seems a lot less intimidating and easier to tackle. Similarly to what Jalynn said, I did not know that my laptop had something called a terminal. The hardest part for me was understanding specific vocabulary and navigating new tools ie. knowing wh

In [150]:
zulip_chat['content'][-1] == "My experience report: Engaging with these new tools and code was very intimidating, especially trying to tackle the instructions on my own. I expected this feeling as this area is unfamiliar to me. The first obstacle was trying to get the student development pack verified. After finally figuring out the VPN issue, I still did not receive a confirmation of approval. After multiple attempts and a lot of feelings of overwhelm and stress, I decided to reach out and attend 1-1 office hours with Jaan. Through this office hour and the guidance of Jaan, it was easier to be more patient with myself, break down instructions given, navigate new tools, and troubleshoot when things didn't run smoothly. Having the guidance of the instructor the assignment seems a lot less intimidating and easier to tackle. Similarly to what Jalynn said, I did not know that my laptop had something called a terminal. The hardest part for me was understanding specific vocabulary and navigating new tools ie. knowing what buttons to click, where to find files etc."

True

In [151]:
zulip_chat["content"][0]

'This is a message on stream #**general** with the topic `topic demonstration`.'

In [152]:
zulip_chat["subject"] == "experience reports"

subject
bool
false
false
false
false
false
false
false
false
false


In [153]:
zulip_chat.filter(pl.col("subject") == "experience reports")

id,sender,recipient,realm,subject,content,rendered_content,rendered_content_version,date_sent,sending_client,last_edit_time,edit_history,has_attachment,has_image,has_link,search_tsvector
u64,u64,i64,u64,str,str,str,u64,str,i64,str,str,bool,bool,bool,str
364697501,544719,924073,47304,"""experience rep…","""i'm confused b…","""<p>i'm confuse…",1,"""1686266940.305…",1163,null,null,true,true,true,"""'/api/views/er…"
364697542,544719,924073,47304,"""experience rep…","""these seem to …","""<p>these seem …",1,"""1686266970.068…",1163,null,null,false,false,true,"""'/github/onefa…"
365383919,622681,924073,47304,"""experience rep…","""Here's my expe…","""<p>Here's my e…",1,"""1686512806.218…",1163,"""1686513322.139…","""[{""user_id"":62…",false,false,false,"""'-2022':244 '2…"
365386362,622681,924073,47304,"""experience rep…","""When i plugged…","""<p>When i plug…",1,"""1686513899.627…",1163,null,null,false,false,false,"""'3':17 'body':…"
365395540,544719,924073,47304,"""experience rep…","""Wow I really a…","""<p>Wow I reall…",1,"""1686519001.548…",1084,null,null,false,false,true,"""'/article/sent…"
365396942,544719,924073,47304,"""experience rep…","""Also just dm’e…","""<p>Also just d…",1,"""1686519974.826…",1084,null,null,false,false,false,"""'also':3 'beha…"
365446290,623389,924073,47304,"""experience rep…","""Hello everyone…","""<p>Hello every…",1,"""1686548959.086…",1000,null,null,false,false,false,"""'3.11':209 'af…"
365459055,622467,924073,47304,"""experience rep…","""Hi all! My exp…","""<p>Hi all! My …",1,"""1686553190.119…",1163,null,null,false,false,false,"""'ad':30 'aim':…"
365547443,622689,924073,47304,"""experience rep…","""Hello everyone…","""<p>Hello every…",1,"""1686573437.469…",1163,null,null,false,false,false,"""'al':164 'also…"


In [154]:
zulip_chat.filter(pl.col("subject") == "experience reports")["content"]

content
str
"""i'm confused b…"
"""these seem to …"
"""Here's my expe…"
"""When i plugged…"
"""Wow I really a…"
"""Also just dm’e…"
"""Hello everyone…"
"""Hi all! My exp…"
"""Hello everyone…"


In [155]:
list_of_all_experience_reports = zulip_chat.filter(pl.col("subject") == "experience reports")["content"].to_list()

In [156]:
list_of_all_experience_reports

['i\'m confused because the instructions i googled were incorrect, and it\'s frustrating! (https://code.visualstudio.com/docs/sourcecontrol/github) and i had to trust a folder but i\'m not sure if that\'s dangerous.\n\ni got so many errors trying to follow the instructions/algorithm: \n\n```\n------------------------------------------\nFileNotFoundError                         Traceback (most recent call last)\nCell In[1], line 2\n      1 import urllib.request\n----> 2 urllib.request.urlretrieve("https://data.cityofnewyork.us/api/views/erm2-nwe9/rows.csv?accessType=DOWNLOAD", "./data/cityofnewyork.us/311-Service-Requests-from-2010-to-Present.csv")\n\nFile /opt/homebrew/anaconda3/envs/datathinking.org-test/lib/python3.11/urllib/request.py:251, in urlretrieve(url, filename, reporthook, data)\n    249 # Handle temporary file setup.\n    250 if filename:\n--> 251     tfp = open(filename, \'wb\')\n    252 else:\n    253     tfp = tempfile.NamedTemporaryFile(delete=False)\n\nFileNotFoundErro

In [157]:
zulip_chat.filter(pl.col("subject") == "experience reports")["content"].apply

<bound method Series.apply of shape: (24,)
Series: 'content' [str]
[
	"i'm confused b…
	"these seem to …
	"Here's my expe…
	"When i plugged…
	"Wow I really a…
	"Also just dm’e…
	"Hello everyone…
	"Hi all! My exp…
	"Hello everyone…
	"@_**Sakariya M…
	"@_**Jane Castl…
	"I actually alr…
	"Here is my exp…
	"I was tasked w…
	"thank you @**D…
	"Here’s my expe…
	"I got overwhel…
	"amazing work @…
	"I tried the ho…
	"This my ChatGP…
	"My experience …
	"@_**Moses Njug…
	"My experience …
	"My experience …
]>

In [158]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/me/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [159]:
# Import stop words list
from nltk.corpus import stopwords 
from collections import Counter

# Remove stop words and tokenize 
message_list = zulip_chat.filter(pl.col("subject") == "experience reports")["content"].apply(lambda msg: [word for word in msg.lower().split() if word not in stopwords.words('english')])

# Flatten list of lists into single list of words 
words = [word for msg in message_list for word in msg]

# Count frequencies 
counts = Counter(words)

# Sort counts and take top 10 
top_words = counts.most_common(10) 

# Create bar chart 
chart = alt.Chart(top_words).mark_bar().encode(
    x='count', 
    y=alt.Y('word', sort='-x')
)

In [160]:
top_words

[('like', 18),
 ('experience', 15),
 ('think', 15),
 ('also', 14),
 ('instructions', 13),
 ('use', 13),
 ('got', 12),
 ('really', 12),
 ('time', 12),
 ('trying', 11)]

In [161]:
# altair chart for plotting top words


In [162]:
counts.most_common(100)

[('like', 18),
 ('experience', 15),
 ('think', 15),
 ('also', 14),
 ('instructions', 13),
 ('use', 13),
 ('got', 12),
 ('really', 12),
 ('time', 12),
 ('trying', 11),
 ('```', 11),
 ('things', 11),
 ('visual', 11),
 ('code', 11),
 ('would', 10),
 ('get', 10),
 ('work', 10),
 ('found', 9),
 ('felt', 9),
 ('studio', 9),
 ("i'm", 8),
 ('one', 8),
 ('students', 8),
 ('see', 8),
 ('feel', 8),
 ('report:', 8),
 ('environment', 8),
 ('office', 8),
 ('hours', 8),
 ('navigate', 8),
 ('still', 7),
 ('right', 7),
 ('help', 7),
 ('able', 7),
 ('getting', 7),
 ('different', 7),
 ('calendar', 7),
 ('always', 7),
 ('chatgpt', 7),
 ('lot', 7),
 ('many', 6),
 ('going', 6),
 ('article', 6),
 ('mcgraw', 6),
 ('it.', 6),
 ('terminal', 6),
 ('thought', 6),
 ('day', 6),
 ('google', 6),
 ('actually', 6),
 ('find', 6),
 ('feeling', 5),
 ('new', 5),
 ('often', 5),
 ('feelings', 5),
 ('support', 5),
 ('even', 5),
 ('vs', 5),
 ('way', 5),
 ('said', 5),
 ('amazing', 5),
 ('went', 5),
 ('could', 5),
 ('something',

In [163]:
import pandas as pd
# create a dataframe from your data
df = pd.DataFrame(counts.most_common(100), columns=['word', 'frequency'])


In [164]:
df

,word,frequency
0,like,18
1,experience,15
2,think,15
3,also,14
4,instructions,13
...,...,...
95,stuck,4
96,last,4
97,made,4
98,plans,4


In [165]:
# create a bar chart
chart = alt.Chart(df).mark_bar().encode(
    y=alt.Y('word:N', title='Words', sort='-x'),
    x=alt.X('frequency:Q', title='Frequency'),
)

In [166]:
chart

alt.Chart(...)

In [167]:
# count the number of words in each message
zulip_chat.filter(pl.col("subject") == "experience reports")["content"].apply(lambda msg: len(msg.split()))

content
i64
155
19
457
16
187
55
248
97
198
